In [ ]:
import sys
sys.path.append('../../../../laddie/src/')
sys.path.append('../../../src2/')

import numpy as np
import xarray as xr
import pandas as pd
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

from laddie2 import Laddie
from geometry3 import Geometry
from forcing import Forcing
from tools import im,jm
from tools import im,jm


%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
exp_class = os.getcwd()[64:-5]
run = os.getcwd()[-4:]
savename = f'{exp_class}_{run}'
print(f'Documents are saved to folder \'{os.getcwd()}\' with save name', savename)

In [ ]:
# Select input file geometry and initial geometry at t = 0
ds_geom = xr.open_dataset('../../../data/Paolo-variablegeom/PI_paolo_30yrs.nc')
ds_t0 = ds_geom.isel(t=0)

# Generate geometry
N = 2                          # standard resolution is at 500 m, N = 2: 1 km, N = 4: 2 km !!! DOES NOT WORK PROPERLY
geom = Geometry(ds_t0, savename)
geom.coarsen(N=N)
geom = geom.create()

# Select forcing
forc = Forcing(geom).isomip('WARM')

In [ ]:
# Set up LADDIE
laddie = Laddie(forc)

laddie.geomname = savename
laddie.N = N
laddie.opt = 1
laddie.calving = True

laddie.Ah = 50               #Horizontal viscosity
laddie.Kh = 50              #Horizontal diffusivity
laddie.slip = .5            #Semi-slip
laddie.convop = 1

laddie.calving = True
laddie.correctisf = True    #Correct ice front
laddie.nnUV = 1             #Weight of nearest neighbours for new cells
laddie.vcut = 1             #maximum velocity

laddie.Cdtop = 1.1e-3       #Top drag coefficient
laddie.minD = 2.8             #Minimum layer thickness
laddie.dt = 180             #Time step
laddie.nu = .5
laddie.gamTfix = None

In [ ]:
# Run with prescribed geometry changes based on Paolo et al. 2022
ds_update = ds_geom.isel(t=slice(1,31))

laddiedays = 1 # number of days to integrate laddie for each ice time step
laddiedayspergeom = 20 # number of days to spin up laddie

laddie.saveday = laddiedays
laddie.restday = 10000

if laddie.opt == 1:
    savename = f'{exp_class}_{run}'
    SY = int(geom.t.values)
    EY = int(ds_update.t.values[-1])
    savename = f'{savename}_UF{laddiedayspergeom:03.0f}_SY{SY:03.0f}_EY{EY:03.0f}'

ds = laddie.compute2(days=laddiedayspergeom, prescribed_geom_ds= ds_update, savename=f'{os.getcwd()}/{savename}')